In [46]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json

In [47]:
# 2020년대 트레이드 내역 데이터 불러오기
with open("player_moves/trade_list_2020s.json") as f:
    trades = json.load(f)

# 2020년데 지명권 트레이드 결과 데이터 불러오기
draft_tickets = pd.read_csv("player_moves/draft_tickets_2020s.csv", keep_default_na=False)

In [48]:
draft_tickets.head(3)

,id,statizId,팀,지명라운드,선수,포지션
0,9,15049,롯데,22-2-3,김세민,내야수
1,10,15027,키움,22-2-4,노운현,투수
2,15,15055,NC,22-2-4,조효원,내야수


In [49]:
player_info_df = [] # 선수 기본정보 크롤링

for trade in trades:
    for x in trade["playerA"] + trade["playerB"]:
        if x["type"] == "money":
            continue

        elif x["type"] == "draft":
            player_id = draft_tickets.loc[(draft_tickets["id"] == trade["id"]) & \
                + (draft_tickets["지명라운드"] == x["round"]), "statizId"].item()
            name = draft_tickets.loc[(draft_tickets["id"] == trade["id"]) & \
                + (draft_tickets["지명라운드"] == x["round"]), "선수"].item()
            
            if player_id == '':
                continue

        else:
            name = x["name"]
            player_id = x["statizId"]

        player_url = f"https://statiz.sporki.com/player/?m=playerinfo&p_no={player_id}"
        driver = webdriver.Chrome()
        driver.get(player_url)
        time.sleep(0.5)

        info1 = tuple(map(lambda x : x.text, driver.find_elements(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.player_info_header > div.bio.type01 > div.in_box > div.p_info > div.con > span")))
        team, position, tuta = info1
        
        info2 = tuple(map(lambda x : x.text[x.text.find('\n')+1:], driver.find_elements(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.player_info_header > div.bio.type01 > div.in_box > ul > li")))
        birth, school, draft, career, team_history = info2

        player_info_df.append([player_id, name, team, position, tuta, birth, school, draft, career, team_history])

        driver.quit()

In [50]:
player_info_df = \
    pd.DataFrame(player_info_df, 
    columns=["statizId", "이름", "팀", "주포지션", "투타", "생년월일", "출신학교", "신인지명", "활약년도", "활약팀"])

player_info_df = player_info_df.drop_duplicates()
player_info_df.to_csv("player_stats/player_basic_info.csv")

In [ ]:
batter_header_url = "https://statiz.sporki.com/player/?m=year&m2=batting&m3=default&p_no=10475&lt=10100&gc="
pitcher_header_url = "https://statiz.sporki.com/player/?m=year&m2=pitching&m3=default&p_no=10475&lt=10100&gc="
fielder_header_url = "https://statiz.sporki.com/player/?m=year&m2=fielding&m3=default&p_no=10475&lt=10100&gc="

# 연도별 - 타격 데이터 헤더 수집
driver = webdriver.Chrome()
driver.get(batter_header_url)
batter_header1 = driver.find_elements(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.table_type02.transverse_scroll.cbox > table > thead:nth-child(1) > tr:nth-child(1) > th")
batter_header2 = driver.find_elements(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.table_type02.transverse_scroll.cbox > table > thead:nth-child(1) > tr:nth-child(2) > th")
batter_header = [x.text for x in (batter_header1[:-2] + batter_header2 + [batter_header1[-1]])]
driver.quit()

# 연도별 - 투구 데이터 헤더 수집
driver = webdriver.Chrome()
driver.get(pitcher_header_url)
pitcher_header = driver.find_elements(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.table_type02.transverse_scroll.cbox > table > thead:nth-child(1) > tr:nth-child(1) > th")
pitcher_header= [x.text for x in pitcher_header]
driver.quit()

# 연도별 - 수비 데이터 헤더 수집
driver = webdriver.Chrome()
driver.get(fielder_header_url)
fielder_header1 = driver.find_elements(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.table_type02.transverse_scroll.cbox > table > thead:nth-child(1) > tr:nth-child(1) > th")
fielder_header2 = driver.find_elements(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.table_type02.transverse_scroll.cbox > table > thead:nth-child(1) > tr:nth-child(2) > th")
fielder_header = [x.text for x in (fielder_header1[:-5] + fielder_header2 + fielder_header1[-4:])]
driver.quit()

In [58]:
player_stat_df = []
batter = ["1B", "2B", "3B", "SS", "LF", "CF", "RF", "C"]

for trade in trades:
    for x in trade["playerA"] + trade["playerB"]:
        if x["type"] == "money":
            continue

        elif x["type"] == "draft":
            player_id = draft_tickets.loc[(draft_tickets["id"] == trade["id"]) & \
                + (draft_tickets["지명라운드"] == x["round"]), "statizId"].item()
            name = draft_tickets.loc[(draft_tickets["id"] == trade["id"]) & \
                + (draft_tickets["지명라운드"] == x["round"]), "선수"].item()
            
            if player_id == '':
                continue

        else:
            name = x["name"]
            player_id = x["statizId"]

        driver = webdriver.Chrome()

        if player_info_df.loc[player_info_df["statizId"] == player_id, "주포지션"].item() in batter:
            batter_stats_url = f"https://statiz.sporki.com/player/?m=year&m2=batting&m3=default&p_no={player_id}&lt=10100&gc="
            fielder_stats_url = f"https://statiz.sporki.com/player/?m=year&m2=fielding&m3=default&p_no={player_id}&lt=10100&gc="

            # 연도별 - 타격 데이터 불러오기
            driver.get(batter_stats_url)
            batter_stats_table = driver.find_element(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.table_type02.transverse_scroll.cbox > table")
            batter_stats_thead = batter_stats_table.find_elements(By.CSS_SELECTOR, "table > thead:nth-child(1) > tr")
            header_1 = batter_stats_thead[0].find_elements(By.CSS_SELECTOR, "tr > th")
            for x in header_1:
                print(x.text)

            # 연도별 - 수비 데이터 불러오기
            
            time.sleep(0.5)

        else:
            pitcher_stats_url = f"https://statiz.sporki.com/player/?m=year&m2=pitching&m3=default&p_no={player_id}&lt=10100&gc="

        

Year
Team
Age
Pos.
oWAR
dWAR
G
PA
ePA
AB
R
H
2B
3B
HR
TB
RBI
SB
CS
BB
HP
IB
SO
GDP
SH
SF
비율
WAR


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF6718680D5+2992373]
	(No symbol) [0x00007FF6714FBFD0]
	(No symbol) [0x00007FF67139590A]
	(No symbol) [0x00007FF67136F4F5]
	(No symbol) [0x00007FF6714163A7]
	(No symbol) [0x00007FF67142EE72]
	(No symbol) [0x00007FF67140F113]
	(No symbol) [0x00007FF6713DA918]
	(No symbol) [0x00007FF6713DBA81]
	GetHandleVerifier [0x00007FF6718C6A2D+3379789]
	GetHandleVerifier [0x00007FF6718DC32D+3468109]
	GetHandleVerifier [0x00007FF6718D0043+3418211]
	GetHandleVerifier [0x00007FF67165C78B+847787]
	(No symbol) [0x00007FF67150757F]
	(No symbol) [0x00007FF671502FC4]
	(No symbol) [0x00007FF67150315D]
	(No symbol) [0x00007FF6714F2979]
	BaseThreadInitThunk [0x00007FFE1FD77374+20]
	RtlUserThreadStart [0x00007FFE203DCC91+33]
